# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [ ]:
#get a list of all song JSON files in data/song_data
song_files= get_files('data/song_data/')

In [ ]:
#gets the first song in the list
filepath= song_files[0]

In [ ]:
#reads the song file and views the data
df = pd.read_json(filepath, lines=True)
df.head()

In [ ]:
#selects only song_id, title, artist_id, year, and duration columns for the first record in the dataframe and converts 
#the array into a list
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

In [ ]:
#insert the record into the songs datatable
cur.execute(song_table_insert, song_data)
conn.commit()

In [ ]:
#select only the artist_id, name, location, latitude, and longitude columns and 
#retrieve the first record in the dataframe. Then we convert the array to a list 
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

In [ ]:
#insert the data in the artists datatable
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [ ]:
#now we get a list of all the log JSON files
log_files = get_files('data/log_data')

In [ ]:
#select the first log file in the list
filepath = log_files[0]

In [ ]:
#read the log file and view data
df = pd.read_json(filepath, lines=True)
df.head()

In [ ]:
#filter records by NextSong action
df = df[df['page']=='NextSong']
df.head()

In [ ]:
#convert ts timestamp column to datetime
t = pd.to_datetime(df['ts'], unit = 'ms')
t.head()

In [ ]:
#extract the timestamp, hour, day, week of year, month, year, and weekday to a list
time_data = [t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
time_dict= dict(zip(column_labels, time_data))

In [ ]:
#create a dataframe with these values
time_df = pd.DataFrame(time_dict)
time_df.head()

In [ ]:
#insert data into time datatable
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [ ]:
#select columns for users datatable
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

In [ ]:
#insert data for users datatable
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [ ]:
#use song_select query to find the song ID and artist ID based on the title, artist name, and duration of a song
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.